In [1]:
%pwd

'f:\\Git_Projects\\Diamon Price Prediction\\notebook'

In [2]:
import os
os.chdir("../")

In [3]:
from dataclasses import dataclass
from pathlib import Path

In [4]:
@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path

In [5]:
from src.mlReg.utils.common import read_yaml, create_directories
from src.mlReg.constants import *

In [6]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = Config_File_Path,
            params_filepath = Params_File_Path,
            schema_filepath = Schema_File_Path
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir= config.root_dir,
            data_path= config.data_path
        )

        return data_transformation_config

In [7]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from src.mlReg.utils.logger import logger

In [8]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
    
    def spliting_data(self):
        data = pd.read_csv(self.config.data_path)

        cat_cols = data.select_dtypes(include= 'object')

        encoding = OrdinalEncoder(dtype= 'int64')

        # Split the data into train and test features.
        train, test = train_test_split(data, test_size=0.2, random_state=42)

        train.to_csv(os.path.join(self.config.root_dir, 'train.csv'), index=False)
        test.to_csv(os.path.join(self.config.root_dir, 'test.csv'), index=False)

        logger.info(f"Split data into training and testing features")
        logger.info(train.shape)
        logger.info(test.shape)

In [9]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config= data_transformation_config)
    data_transformation.spliting_data()
except Exception as e:
    raise e

[ [2024-03-07 15:38:38,566] : INFO : common : yaml file: config\config.yaml loaded successfully ]
[ [2024-03-07 15:38:38,582] : INFO : common : yaml file: params.yaml loaded successfully ]
[ [2024-03-07 15:38:38,590] : INFO : common : yaml file: schema.yaml loaded successfully ]
[ [2024-03-07 15:38:38,590] : INFO : common : created directory at: artifacts ]
[ [2024-03-07 15:38:38,598] : INFO : common : created directory at: artifacts/data_transformation ]
[ [2024-03-07 15:38:42,242] : INFO : 2822314415 : Split data into training and testing features ]
[ [2024-03-07 15:38:42,250] : INFO : 2822314415 : (154858, 11) ]
[ [2024-03-07 15:38:42,250] : INFO : 2822314415 : (38715, 11) ]
